In [17]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [18]:
df = pd.read_csv("/workspace/KordSADLPython/Documents/data/darts.csv")
df.head()

,xCoord,yCoord,competitor
0,0.196451,-0.520341,Steve
1,0.476027,-0.306763,Susan
2,0.003175,-0.980736,Michael
3,0.294078,0.267566,Kate
4,-0.051120,0.598946,Steve


In [19]:
df["competitor"].value_counts()

competitor
Steve      199
Susan      199
Kate       199
Michael    198
Name: count, dtype: int64

In [20]:
df.describe()

,xCoord,yCoord
count,795.000000,795.000000
mean,0.006288,0.022780
std,0.476283,0.513699
min,-0.991140,-0.993547
25%,-0.295523,-0.424348
50%,0.000619,0.034675
75%,0.271990,0.449876
max,0.984816,0.994344


In [21]:
from tensorflow.keras.utils import to_categorical
pd.Categorical(df.competitor).codes
to_categorical(pd.Categorical(df.competitor).codes)

array([[0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       ...,
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]], dtype=float32)

In [22]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
enc = OneHotEncoder()
enc.fit_transform(df[["competitor"]]).toarray()

array([[0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       ...,
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]])

In [23]:
enc.fit_transform(df[["competitor"]]).toarray()

array([[0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       ...,
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]])

#### Modelin Oluşturulması

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()


model.add(Dense(128,input_shape=(2,),activation="relu"))
model.add(Dense(64,activation="relu"))
model.add(Dense(32,activation="relu"))

model.add(Dense(4,activation="softmax"))

model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])


In [25]:
y = df.competitor.values
X = df.drop("competitor",axis=1).values

In [26]:
from tensorflow.keras.utils import to_categorical
# pd.Categorical(y).codes
y = to_categorical(pd.Categorical(y).codes)

In [27]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,stratify=y,shuffle=True,random_state=42)

In [28]:
model.fit(X_train,y_train,epochs=400,batch_size=25)

Epoch 1/400
26/26 [==============================] - 1s 2ms/step - loss: 1.3744 - accuracy: 0.2610
Epoch 2/400
26/26 [==============================] - 0s 2ms/step - loss: 1.3245 - accuracy: 0.3396
Epoch 3/400
26/26 [==============================] - 0s 2ms/step - loss: 1.2440 - accuracy: 0.4292
Epoch 4/400
26/26 [==============================] - 0s 2ms/step - loss: 1.1210 - accuracy: 0.5063
Epoch 5/400
26/26 [==============================] - 0s 1ms/step - loss: 0.9871 - accuracy: 0.5770
Epoch 6/400
26/26 [==============================] - 0s 1ms/step - loss: 0.9083 - accuracy: 0.6022
Epoch 7/400
26/26 [==============================] - 0s 1ms/step - loss: 0.8710 - accuracy: 0.6305
Epoch 8/400
26/26 [==============================] - 0s 1ms/step - loss: 0.8285 - accuracy: 0.6714
Epoch 9/400
26/26 [==============================] - 0s 1ms/step - loss: 0.8112 - accuracy: 0.6619
Epoch 10/400
26/26 [==============================] - 0s 1ms/step - loss: 0.7954 - accuracy: 0.7044
Epoch 11/

In [29]:
model.evaluate(X_test,y_test)[1]

5/5 [==============================] - 0s 1ms/step - loss: 0.7356 - accuracy: 0.8050


0.805031418800354

In [30]:
# 0.320434866200987,-0.327131082296809,Kate
# -0.179371803861741,0.59420054627762,Steve
# 0.530314557940464,-0.83332095816775,Michael
# -0.287257514007552,0.890927793559789,Michael
# -0.628402820141772,-0.140867602052845,Susan

In [31]:
real_test = np.array([[0.320434866200987,-0.327131082296809],
[-0.179371803861741,0.59420054627762],
[0.530314557940464,-0.83332095816775],
[-0.287257514007552,0.890927793559789],
[-0.628402820141772,-0.140867602052845]])

In [32]:
preds = model.predict(real_test)
for i,preds in enumerate(preds):
    print(f"{preds}----------|------------{real_test[i]}")

1/1 [==============================] - 0s 46ms/step
[0.8956553  0.01578094 0.0868867  0.001677  ]----------|------------[ 0.32043487 -0.32713108]
[0.9292229  0.00440371 0.06196519 0.00440814]----------|------------[-0.1793718   0.59420055]
[3.0262519e-03 9.6866155e-01 2.8311564e-02 6.5758411e-07]----------|------------[ 0.53031456 -0.83332096]
[0.06780989 0.89575905 0.00413639 0.03229471]----------|------------[-0.28725751  0.89092779]
[3.4603876e-01 4.6459779e-02 1.0857585e-04 6.0739297e-01]----------|------------[-0.62840282 -0.1408676 ]
